#Park detection project


## Load dataset

In [ ]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="REItrMfvW0zkmeutbv41")
project = rf.workspace("capping-project").project("lot-spot")
version = project.version(2)
dataset = version.download("yolov8-obb")


##Prepare the dataset

In [1]:
# relabel an image incorrectly labeled

import os

incorrect_image_path = "/home/raffaele/park_detection/lot-spot-2/train/labels/lot_jpg.rf.01dbf19416b625ad24bd92f6fe2f0682.txt"

with open(incorrect_image_path, 'r') as file:
    lines = file.readlines()
    modified_lines = []
    for line in lines:
        if line.startswith('2'):
            modified_lines.append(line.replace('2', '1', 1))
        elif line.startswith('3'):
            modified_lines.append(line.replace('3', '0', 1))


with open(incorrect_image_path, 'w') as file:
    file.writelines(modified_lines)

In [2]:
import yaml

dataset_path = "/home/raffaele/park_detection/lot-spot-2"
classes = ['Occupied', 'Not occupied']
nc = len(classes)
config_yaml = {

               "test" : f"{dataset_path}/test",
               "train" : f"{dataset_path}/train",
               "val" : f"{dataset_path}/valid",
               "nc": nc,
               "names": classes,
}
with open(f"{dataset_path}/data.yaml", "w") as file:
  yaml.dump(config_yaml, file)

##Load the model

In [ ]:
!pip install ultralytics

In [3]:
from ultralytics import YOLO

model = YOLO("yolo11n-obb.pt")

##Train the model



First we train the basic one model (with the default hyper-parameters) then we choose the most appropriate combination.

In [ ]:
model.train(data=f'{dataset_path}/data.yaml', epochs=100, imgsz=640, device='0')

In [ ]:
results = model.val(data=f'{dataset_path}/data.yaml', imgsz=640, device='0', split='test')

##Hyper-parameters tuning

In [ ]:
model.tune(data=f'{dataset_path}/data.yaml', imgsz=640, epochs=30, iterations=100, optimizer="AdamW", plots=False, save=False, val=False, verbose=False)

In [ ]:
cfg = '/home/raffaele/park_detection/runs/obb/tune8/best_hyperparameters.yaml'
model.train(data=f'{dataset_path}/data.yaml', epochs=100, imgsz=640, device='0', 
            cfg='/home/raffaele/park_detection/runs/obb/tune8/best_hyperparameters.yaml', optimizer='AdamW', seed=42)

In [ ]:
results = model.val(data=f'{dataset_path}/data.yaml', imgsz=640, device='0', split='test')

In [3]:
batch = [16, 32]
dropout = [0, 0.2, 0.5]
freeze = [None, 5, 7]

In [4]:
import itertools

param_grid = list(itertools.product(batch, dropout, freeze)) 

In [ ]:
model.metrics.box.map

In [ ]:
best_map = 0
best_params = None
for b, d, f in param_grid:
    model = YOLO("yolo11n-obb.pt")
    model.train(data=f'{dataset_path}/data.yaml', epochs=10, imgsz=640, 
           cfg=cfg,
           optimizer='AdamW', device=0, batch=b, dropout=d, freeze=f, seed=42)
    
    # 4) `results` object usually holds metrics or paths to logs
    current_map = model.metrics.box.map    # or .box.map50, .seg.map, etc., depending on your task
    
    # 5) Keep track of the best run
    if current_map > best_map:
        best_map = current_map
        best_params = [b, d, f]

    print(f"  => mAP: {current_map:.4f}")
    print(best_params)

    
    


##Training of the final model

In [5]:
model_n = model = YOLO("yolo11n-obb.pt")
model_n.train(data=f'{dataset_path}/data.yaml', epochs=200, imgsz=640, 
           cfg=cfg,
           optimizer='AdamW', device=0, freeze=None, dropout=0.0, batch=32, seed=42)

    136/200      5.11G     0.4976     0.2779     0.6913        595        640: 100%|██████████| 26/26 [00:06<00:00,  4.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  7.33it/s]

                   all        236       4443      0.982      0.978      0.994      0.958



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    137/200      5.63G     0.4993     0.2846     0.6888        594        640: 100%|██████████| 26/26 [00:06<00:00,  4.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  7.41it/s]

                   all        236       4443      0.974      0.981      0.993      0.956


...

                   all        236       4443      0.983      0.984      0.994      0.966



200 epochs completed in 0.434 hours.
Optimizer stripped from runs/obb/train251/weights/last.pt, 5.9MB
Optimizer stripped from runs/obb/train251/weights/best.pt, 5.9MB

Validating runs/obb/train251/weights/best.pt...
Ultralytics 8.3.57 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7933MiB)
YOLO11n-obb summary (fused): 257 layers, 2,654,113 parameters, 0 gradients, 6.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.32it/s]


                   all        236       4443      0.983      0.983      0.994      0.965
              Occupied        195       2914      0.986      0.985      0.994      0.983
          Not occupied        222       1529      0.979      0.982      0.994      0.948
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/obb/train251


ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x75754f19e2d0>
curves: []
curves_results: []
fitness: 0.9681628285703897
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.98301,     0.94758])
names: {0: 'Occupied', 1: 'Not occupied'}
plot: True
results_dict: {'metrics/precision(B)': 0.9826908955939498, 'metrics/recall(B)': 0.9833172710976164, 'metrics/mAP50(B)': 0.9939745022973634, 'metrics/mAP50-95(B)': 0.9652948648229481, 'fitness': 0.9681628285703897}
save_dir: PosixPath('runs/obb/train251')
speed: {'preprocess': 0.19419698391930532, 'inference': 1.6480219566215903, 'loss': 0.001155723959712659, 'postprocess': 1.5727665464756853}

In [9]:
model_n = YOLO('runs/obb/train251/weights/best.pt')
model_n.val(split='test', data=f'{dataset_path}/data.yaml', seed=42)

Ultralytics 8.3.57 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7933MiB)
YOLO11n-obb summary (fused): 257 layers, 2,654,113 parameters, 0 gradients, 6.6 GFLOPs


val: Scanning /home/raffaele/park_detection/lot-spot-2/test/labels.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.75it/s]


                   all        118       2665      0.987      0.986      0.994      0.972
              Occupied        107       1879      0.993      0.991      0.995      0.986
          Not occupied        108        786      0.981       0.98      0.994      0.958
Speed: 1.2ms preprocess, 2.5ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/obb/val4


ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7120a242cc80>
curves: []
curves_results: []
fitness: 0.9743999941560977
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.98641,     0.95793])
names: {0: 'Occupied', 1: 'Not occupied'}
plot: True
results_dict: {'metrics/precision(B)': 0.9869349967081256, 'metrics/recall(B)': 0.9855642991304725, 'metrics/mAP50(B)': 0.9944660797393923, 'metrics/mAP50-95(B)': 0.9721704290912871, 'fitness': 0.9743999941560977}
save_dir: PosixPath('runs/obb/val4')
speed: {'preprocess': 1.1908260442442813, 'inference': 2.5110083111261914, 'loss': 0.0029014328778800315, 'postprocess': 2.0740476705260193}

##Export model

In [11]:
model_n.export(format="onnx")

Ultralytics 8.3.57 🚀 Python-3.12.7 torch-2.5.1+cu124 CPU (13th Gen Intel Core(TM) i9-13900HX)

PyTorch: starting from 'runs/obb/train251/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.6 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.47...
ONNX: export success ✅ 0.6s, saved as 'runs/obb/train251/weights/best.onnx' (10.3 MB)

Export complete (0.8s)
Results saved to /home/raffaele/park_detection/runs/obb/train251/weights
Predict:         yolo predict task=obb model=runs/obb/train251/weights/best.onnx imgsz=640  
Validate:        yolo val task=obb model=runs/obb/train251/weights/best.onnx imgsz=640 data=/home/raffaele/park_detection/lot-spot-2/data.yaml  
Visualize:       https://netron.app


'runs/obb/train251/weights/best.onnx'

In [13]:
model_n = YOLO('runs/obb/train251/weights/best.onnx')
model_n.val(split='test', data=f'{dataset_path}/data.yaml', seed=42, device='cpu')

Ultralytics 8.3.57 🚀 Python-3.12.7 torch-2.5.1+cu124 CPU (13th Gen Intel Core(TM) i9-13900HX)
Loading runs/obb/train251/weights/best.onnx for ONNX Runtime inference...
requirements: Ultralytics requirement ['onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 5.0 MB/s eta 0:00:0000:0100:01

requirements: AutoUpdate success ✅ 5.3s, installed 1 package: ['onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Using ONNX Runtime CPUExecutionProvider
Setting batch=1 input of shape (1, 3, 640, 640)


val: Scanning /home/raffaele/park_detection/lot-spot-2/test/labels.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 118/118 [00:03<00:00, 31.78it/s]


                   all        118       2665      0.984      0.984      0.994      0.967
              Occupied        107       1879      0.989      0.989      0.995      0.985
          Not occupied        108        786      0.979      0.978      0.993       0.95
Speed: 0.6ms preprocess, 19.4ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to runs/obb/val6


ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x71206e1ba690>
curves: []
curves_results: []
fitness: 0.9699885073760856
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.98495,     0.94969])
names: {0: 'Occupied', 1: 'Not occupied'}
plot: True
results_dict: {'metrics/precision(B)': 0.9843811153325686, 'metrics/recall(B)': 0.9837775643350102, 'metrics/mAP50(B)': 0.9940088274055603, 'metrics/mAP50-95(B)': 0.9673195829283661, 'fitness': 0.9699885073760856}
save_dir: PosixPath('runs/obb/val6')
speed: {'preprocess': 0.5860651953745697, 'inference': 19.43454095872782, 'loss': 0.0009395308413748014, 'postprocess': 4.565711748802056}